## When countries don't match

In this notebook we will focus on the situation when the county of the card doesn't match with the country of the IP

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import researchpy as rp
%matplotlib inline
%load_ext autoreload
%autoreload

df = pd.read_csv('adyen-dataset.csv')

In [3]:
df['same_country'] = df['ip_country'] == df['issuing_country']
df.head()

,psp_reference,merchant,card_scheme,year,hour_of_day,minute_of_hour,day_of_year,is_credit,eur_amount,ip_country,...,device_type,ip_address,email_address,card_number,shopper_interaction,zip_code,card_bin,has_fraudulent_dispute,is_refused_by_adyen,same_country
0,79622457320,Merchant A,Other,2021,13,51,206,True,18.75,GR,...,MacOS,i1oY1kTOXMlsFfoyhFJxyA,3XBRYto3QgX7g4pX-BvNyA,XldVNN9TQgIA2RE9FTC2TA,Ecommerce,XDED,4236,False,False,True
1,41293916020,Merchant C,Visa,2021,15,20,345,False,43.65,BR,...,Android,HZkxQvtpScNLgi0fsoSkjA,klwgzdfrnz_6eGsHSi2prQ,zgRoL8pxX5LJI-SEPUTMvg,Ecommerce,AAD,4802,False,False,True
2,31382555255,Merchant E,Visa,2021,1,56,293,True,187.72,NL,...,MacOS,HGW5EwBnHVgOrD5xYplBdw,XkMWeLLmXALPcV3ndwzKJw,T3BF1uOOqgcwaVRb_ujPQg,Ecommerce,1104,4526,False,False,True
3,13944051314,Merchant D,Visa,2021,12,50,242,False,109.16,NL,...,iOS,8AuFEjK8m-0lxGAOJzOPmg,26Lw20NQtPKb5cyQZBuh1w,BMf1zFkJ1xvPMlOxC7RWTw,Ecommerce,2039,4571,False,True,True
4,62343575411,Merchant E,MasterCard,2021,19,26,318,True,46.27,BR,...,Other,NaN,Y7tv2i85iUadkEIAPIufgg,jfxS2TIe0gR9vpOUdMKrBw,Ecommerce,XOL,4916,False,False,True


In [4]:
from lib.correlation import calculate_woe_iv
df_woe = pd.crosstab(index = df['same_country'], columns=df["has_fraudulent_dispute"])
df_woe.rename(columns={0:"neg", 1:"pos"}, inplace=True)
df_woe = calculate_woe_iv(df_woe)
df_woe[1]

has_fraudulent_dispute,neg,pos,pos_rate,neg_rate,woe,iv
same_country,,,,,,
False,20402,3361,0.313736,0.159417,0.677029,0.104479
True,107583,7355,0.686450,0.840599,-0.202581,0.031227


We can see how same_country == False is more likely to be fraud

### Demographic parity in same_country

It could happen that in some countries having a different country IP is quite common, so instead of being 0/1 we will try on assigining a continuous variable for each country in case of same_country == False

In [5]:
scFalse = df.groupby(df["issuing_country"]).apply(lambda x: x[x['same_country'] == False].count()[['same_country']])
scTrue  = df.groupby(df["issuing_country"]).count()[['same_country']]

dfCountries = pd.merge(scFalse, scTrue, left_index=True, right_index=True)
dfCountries["Proportion"] = dfCountries["same_country_x"] / dfCountries["same_country_y"]
dfCountries

,same_country_x,same_country_y,Proportion
issuing_country,,,
BR,5344,30266,0.176568
GR,4943,28826,0.171477
IT,2333,10802,0.215979
NL,6362,39971,0.159165
ZW,4781,28836,0.165800


In [6]:
crosstab, test_results, expected = rp.crosstab(df["issuing_country"], df["same_country"],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

expected

same_country              
same_country           False          True
issuing_country                           
BR               5185.333617  25080.666383
GR               4938.625086  23887.374914
IT               1850.656635   8951.343365
NL               6848.046323  33122.953677
ZW               4940.338339  23895.661661

In [7]:
df_filtered = df.loc[(df['issuing_country'] == "NL") | (df['issuing_country'] == "IT")]
crosstab, test_results, expected = rp.crosstab(df_filtered["issuing_country"], df_filtered["same_country"],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

expected

same_country              
same_country           False          True
issuing_country                           
IT               1849.868828   8952.131172
NL               6845.131172  33125.868828

#### Adjusting Same_Country rate

In [8]:
scFalse = df.groupby(df["issuing_country"]).apply(lambda x: x[x['same_country'] == False].count()[['same_country']])
scTrue  = df.groupby(df["issuing_country"]).count()[['same_country']]

dfCountries = pd.merge(scFalse, scTrue, left_index=True, right_index=True)
dfCountries["Proportion"] = dfCountries["same_country_x"] / dfCountries["same_country_y"]
display(dfCountries)


crosstab, test_results, expected = rp.crosstab(df["ip_country"], df["same_country"],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

display(expected)



,same_country_x,same_country_y,Proportion
issuing_country,,,
BR,5344,30266,0.176568
GR,4943,28826,0.171477
IT,2333,10802,0.215979
NL,6362,39971,0.159165
ZW,4781,28836,0.165800


same_country              
same_country        False          True
ip_country                             
BR            5179.165904  25050.834096
GR            4937.254483  23880.745517
IT            1867.960498   9035.039502
NL            6824.060771  33006.939229
ZW            4954.558345  23964.441655

In [9]:
exp_obs = expected.join(dfCountries)[[('same_country', False), "same_country_x"]]
exp_obs.columns = ["Expected", "Observed"]
exp_obs["ConversionRatio"] = exp_obs["Expected"]/exp_obs["Observed"]
display(exp_obs[["ConversionRatio"]])
dictRatios = exp_obs[["ConversionRatio"]].to_dict()

c:\Users\aguil\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(
c:\Users\aguil\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


,ConversionRatio
ip_country,
BR,0.969155
GR,0.998838
IT,0.800669
NL,1.072628
ZW,1.036302


In [9]:
df_test = pd.read_csv('adyen-dataset.csv')

df_test["same_country"] = (df_test["ip_country"] != df_test["issuing_country"]).astype(int) * df_test["issuing_country"].map(dictRatios['ConversionRatio'])

scFalse = df_test.groupby(df_test["issuing_country"]).sum()[['same_country']]
scTrue  = df_test.groupby(df_test["issuing_country"]).count()[['same_country']]

dfCountries = pd.merge(scFalse, scTrue, left_index=True, right_index=True)
dfCountries["Proportion"] = dfCountries["same_country_x"] / dfCountries["same_country_y"]
display(dfCountries)

,same_country_x,same_country_y,Proportion
issuing_country,,,
BR,5179.165904,30266,0.171122
GR,4937.254483,28826,0.171278
IT,1867.960498,10802,0.172927
NL,6824.060771,39971,0.170725
ZW,4954.558345,28836,0.171819


#### Adjusting Same_Country rate multiple combinations

What if there is combinations of different countries that are likely to be legal. This should be considered as well.

In [10]:
df = pd.read_csv('adyen-dataset.csv')

In [16]:
cols = df["issuing_country"].unique()
cols = cols.tolist()

con_table = pd.DataFrame(columns=np.append(["issuing_country"], cols))

for issuing_c in cols:
    dict = {}
    for ip_c in cols:
        value = len(df.loc[(df["issuing_country"] == issuing_c) & (df["ip_country"] == ip_c)])
        dict[ip_c] = value
    dict["issuing_country"] = issuing_c
    con_table = con_table.append(dict, ignore_index=True)
    
con_table = con_table.set_index("issuing_country")
con_table

,GR,BR,NL,ZW,IT
issuing_country,,,,,
GR,23883,1387,1795,1254,507
BR,1389,24922,1974,1367,614
NL,1794,2017,33609,1753,798
ZW,1253,1316,1697,24055,515
IT,499,588,756,490,8469


In [12]:
total_rows = con_table.sum(axis=1).to_dict()
total_rows

{'GR': 28826.0, 'BR': 30266.0, 'NL': 39971.0, 'ZW': 28836.0, 'IT': 10802.0}

In [25]:
cols = df["issuing_country"].unique()
cols = cols.tolist()

con_table_fraud = pd.DataFrame(columns=np.append(["issuing_country"], cols))

for issuing_c in cols:
    dict = {}
    for ip_c in cols:
        value = len(df.loc[(df["issuing_country"] == issuing_c) & (df["ip_country"] == ip_c) & (df["has_fraudulent_dispute"] == True)])
        dict[ip_c] = value
    dict["issuing_country"] = issuing_c
    con_table_fraud = con_table_fraud.append(dict, ignore_index=True)
    
con_table_fraud = con_table_fraud.set_index("issuing_country")
con_table_fraud.columns = [col + "_fraud" for col in cols]
con_table_fraud

,GR_fraud,BR_fraud,NL_fraud,ZW_fraud,IT_fraud
issuing_country,,,,,
GR,988,165,190,106,91
BR,174,2157,371,175,171
NL,187,345,2217,174,195
ZW,106,147,162,962,108
IT,83,156,176,79,1031


(Quick experiment) Fraud Rate per Country

In [41]:
con_table_fraud.sum(axis=1) / con_table.sum(axis=1)
#Very imbalanced data :(

issuing_country
GR    0.053424
BR    0.100707
NL    0.078007
ZW    0.051498
IT    0.141178
dtype: float64

In [32]:
fraud_rate = pd.concat([con_table, con_table_fraud], axis=1)

cols_ratio = [col + "_ratio" for col in cols]
cols_fraud = [col + "_fraud" for col in cols]

for i in range(len(cols)):
    fraud_rate[cols_ratio[i]] = fraud_rate[cols_fraud[i]] / fraud_rate[cols[i]]

# What the for-loop is doing:
#fraud_rate["GR_ratio"] = fraud_rate["GR_fraud"] / fraud_rate["GR"]
#fraud_rate["BR_ratio"] = fraud_rate["BR_fraud"] / fraud_rate["BR"]
#fraud_rate["NL_ratio"] = fraud_rate["NL_fraud"] / fraud_rate["NL"]
#fraud_rate["ZW_ratio"] = fraud_rate["ZW_fraud"] / fraud_rate["ZW"]
#fraud_rate["IT_ratio"] = fraud_rate["IT_fraud"] / fraud_rate["IT"]

fraud_rate = fraud_rate[cols_ratio]
fraud_rate

,GR_ratio,BR_ratio,NL_ratio,ZW_ratio,IT_ratio
issuing_country,,,,,
GR,0.041368,0.118962,0.10585,0.08453,0.179487
BR,0.12527,0.08655,0.187943,0.128018,0.278502
NL,0.104236,0.171046,0.065964,0.099258,0.244361
ZW,0.084597,0.111702,0.095463,0.039992,0.209709
IT,0.166333,0.265306,0.232804,0.161224,0.121738


If we used this table as FE it wouldn't be fair anymore. A right/common behavior (countries matching) in countries like Italy would be punished only because they have a higher fraud rate. Because of this, this feature will be calculated based on the differences of countries of same country/different country

In [33]:
fraud_rate["base"] = fraud_rate.min(axis = 1)

cols_fraud_feature = [col + "_fraud_feature" for col in cols]

for i in range(len(cols)):
    fraud_rate[cols_fraud_feature[i]] = (fraud_rate[cols_ratio[i]] - fraud_rate["base"]) / fraud_rate["base"]

fraud_rate = fraud_rate[cols_fraud_feature]
fraud_rate


,GR_fraud_feature,BR_fraud_feature,NL_fraud_feature,ZW_fraud_feature,IT_fraud_feature
issuing_country,,,,,
GR,0.0,1.875672,1.55871,1.043338,3.338757
BR,0.447371,0.0,1.171498,0.479116,2.217811
NL,0.580189,1.593003,0.0,0.504725,2.704432
ZW,1.115364,1.793134,1.387061,0.0,4.243808
IT,0.366316,1.179319,0.912337,0.324355,0.0
